In [1]:
import numpy as np
import pandas as pd

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
region = "EU"

In [3]:
# read IEA data    
file_path = "inputs/IEA Global EV Data 2024.csv"
outlook = pd.read_csv(file_path)
outlook.drop('unit', axis=1, inplace=True)
outlook.set_index(['category','parameter','mode', 'powertrain','region','year'],inplace=True)
outlook_product = pd.pivot_table(outlook, values='value', index=['category','parameter','region','year'], columns=['mode', 'powertrain'], aggfunc='sum')
#for manufacturing phase, we care about the sales of new vehicles, not the whole population
outlook_sales = outlook_product[(outlook_product.index.get_level_values("category")!="Historical")&
                                 (outlook_product.index.get_level_values("parameter")=="EV sales")]

In [4]:
outlook_sales.index.get_level_values("region").unique()

Index(['China', 'Europe', 'India', 'Rest of the world', 'USA', 'World'], dtype='object', name='region')

In [5]:
ev_list = ["PHEV","BEV","FCEV"]
outlook_sales_EU = outlook_sales[outlook_sales.index.get_level_values("region")== "Europe"]["Cars"][ev_list]
outlook_sales_EU.reset_index(level='parameter', drop=True, inplace=True)


In [6]:
outlook_shares = outlook_product[(outlook_product.index.get_level_values("category")!="Historical")&
                                 (outlook_product.index.get_level_values("parameter")=="EV sales share")]
outlook_shares_EU = outlook_shares[outlook_shares.index.get_level_values("region")== "Europe"]["Cars"]["EV"].to_frame()
outlook_shares_EU.reset_index(level='parameter', drop=True, inplace=True)
outlook_shares_EU["ICEV"] = 100 - outlook_shares_EU["EV"]
outlook_shares_EU = outlook_shares_EU/100

outlook_shares_EU

EV  ICEV
category         region year            
Projection-APS   Europe 2020  0.10  0.90
                        2021  0.17  0.83
                        2022  0.20  0.80
                        2023  0.21  0.79
                        2025  0.31  0.69
                        2030  0.62  0.38
                        2035  0.92  0.08
Projection-STEPS Europe 2020  0.10  0.90
                        2021  0.17  0.83
                        2022  0.20  0.80
                        2023  0.21  0.79
                        2025  0.31  0.69
                        2030  0.60  0.40
                        2035  0.87  0.13

In [7]:
outlook_EVsales_EU = outlook_sales_EU.sum(axis=1).to_frame()
outlook_EVsales_EU.rename(columns={0: 'EV'}, inplace=True)

outlook_sales_EU["ICEV"] = (outlook_shares_EU["ICEV"]/outlook_shares_EU["EV"])*outlook_EVsales_EU["EV"]
outlook_sales_EU["ICEV"].round().astype(int)
outlook_sales_EU

powertrain                         PHEV         BEV     FCEV          ICEV
category         region year                                              
Projection-APS   Europe 2020   640000.0    760000.0    740.0  1.260666e+07
                        2021  1100000.0   1200000.0    960.0  1.123410e+07
                        2022  1100000.0   1600000.0   1300.0  1.080520e+07
                        2023  1100000.0   2200000.0    820.0  1.241737e+07
                        2025  1100000.0   3600000.0   5800.0  1.047420e+07
                        2030  1900000.0   7400000.0  10000.0  5.706129e+06
                        2035    35000.0  14000000.0  28000.0  1.222870e+06
Projection-STEPS Europe 2020   640000.0    760000.0    740.0  1.260666e+07
                        2021  1100000.0   1200000.0    960.0  1.123410e+07
                        2022  1100000.0   1600000.0   1300.0  1.080520e+07
                        2023  1100000.0   2200000.0    820.0  1.241737e+07
                        2025  1500000.0   3300000.0   3800.0  1.069233e+07
                        2030  2400000.0   6800000.0   4900.0  6.136600e+06
                        2035   620000.0  13000000.0  10000.0  2.036667e+06

In [9]:
outlook_sales_EU[["ICEV","PHEV","BEV","FCEV"]].to_csv("data/outlook_vehicles_sales.csv")